In [55]:
from youtube_transcript_api import YouTubeTranscriptApi

ImportError: attempted relative import with no known parent package

In [35]:
def get_video_summary(video_transcript):
   return video_transcript

In [46]:
def get_video_transcript(video_id):
    transcript_list = YouTubeTranscriptApi.get_transcript(video_id)
    transcript_text = ' '.join(transcript['text'] for transcript in transcript_list)
    return transcript_text

In [47]:
def get_video_id(video_link):
   return video_link

In [50]:
def main():
   video_link = 'qUmZG3yN_L4'
   video_id = get_video_id(video_link)
   video_transcript = get_video_transcript(video_id)
   video_sumamry = get_video_summary(video_transcript)
   return video_sumamry

In [51]:
if __name__ == "__main__":
   print(main())

tickets are on sale for all my upcoming standup dates Detroit Atlantic City Niagara Falls Mount Pleasant Michigan West Valley City AKA Salt Lake City Utah Reno Nevada Anaheim California and Las Vegas Nevada tickets for sale now for my standup comedy featuring some of your favorite characters from the show especially me all tickets are at Tony Hench cliff.com right now [Applause] [Music] and I One More Time [Applause] [Music] [Applause] [Music] hey this is RIT coming to you live from the Cy Mothership here in Austin Texas for a brand new episode of kill Tony give it up for a Tony let's go who's ready for the best [ __ ] night of their lives [Music] H makes so nice for Brian Red Band ladies and gentlemen and indeed one more time for the best Stam band in the land that's the kill Tony Band everybody brought to you by VA and blue Chu that is indeed groov line horns Fernando Castillo Raul Vallejo and Carlos Sosa nachos Bel Grande that is Big Mike Michael Gonzalez on the drums they came here